In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=pd.read_csv('/content/drive/MyDrive/iot-dataset/device_6_dataset.csv')


In [4]:
data.head()
data.groupby('type')['type'].count()

type
benign             98514
gafgyt_combo       57530
gafgyt_junk        29068
gafgyt_scan        28397
gafgyt_tcp         89387
gafgyt_udp        104658
mirai_ack          57997
mirai_scan         97096
mirai_syn          61851
mirai_udp         158608
mirai_udpplain     53785
Name: type, dtype: int64

In [5]:
data["type"].replace({"gafgyt_combo": "attack" ,
                             "gafgyt_junk" : "attack",
                             "gafgyt_scan" : "attack",
                             "gafgyt_tcp" : "attack",
                             "gafgyt_udp" : "attack",
                             "mirai_ack" : "attack",
                             "mirai_scan" : "attack",
                             "mirai_syn" : "attack",
                             "mirai_udp" : "attack",
                             "mirai_udpplain" : "attack"},
                             inplace=True)

In [6]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [ ]:
#standardize numerical columns
# def standardize(df,col):
#     df[col]= (df[col]-df[col].mean())/df[col].std()
# data_st=data.copy()
# for i in (data_st.iloc[:,:-1].columns):
#     standardize (data_st,i)
# data_st.head()

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=True)

In [9]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score

In [11]:
from keras import Sequential
from keras.layers import Dense

classifier = Sequential()

In [12]:
#First Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal', input_dim=115))
#Second  Hidden Layer
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))

classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))

#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [13]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                7424      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 10,049
Trainable params: 10,049
Non-trainable params: 0
_________________________________________________________________


In [14]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=1)

20923/20923 [==============================] - 44s 2ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 0.0012 - val_accuracy: 0.9997


In [15]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [16]:
print(f"Accuracy   : {accuracy_score(y_test, y_pred)}")
print(f"Precision  : {precision_score(y_test, y_pred)}")
print(f"F1-Measure : {f1_score(y_test, y_pred)}")
print(f"MCC        : {matthews_corrcoef(y_test, y_pred)}")
print(f"Recall     : {recall_score(y_test, y_pred)}")

Accuracy   : 0.9996773788826555
Precision  : 0.9995438649840352
F1-Measure : 0.9986328421692238
MCC        : 0.9984504857792119
Recall     : 0.9977234785248141


In [17]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    147612
           1       1.00      1.00      1.00     19767

    accuracy                           1.00    167379
   macro avg       1.00      1.00      1.00    167379
weighted avg       1.00      1.00      1.00    167379



In [18]:
print(confusion_matrix(y_test, y_pred))

[[147603      9]
 [    45  19722]]
